<a href="https://colab.research.google.com/github/BatuhanSimsk/BatuhanSimsk/blob/main/NST_Latest_Version_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Imports*

In [1]:
from tqdm import tqdm

from google.colab import files

import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

import matplotlib.pyplot as plt

!pip install gradio

import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.2 MB/s

# *VGG19 Neural Model Specialization*

In [2]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        # The first number x in convx_y gets added by 1 after it has gone
        # through a maxpool, and the second y if we have several conv layers
        # in between a max pool. These strings (0, 5, 10, ..) then correspond
        # to conv1_1, conv2_1, conv3_1, conv4_1, conv5_1 mentioned in NST paper
        self.chosen_features = ["0", "5", "10", "19", "28"]

        # We don't need to run anything further than conv5_1 (the 28th module in vgg)
        # Since remember, we dont actually care about the output of VGG: the only thing
        # that is modified is the generated image (i.e, the input).
        self.model = models.vgg19(pretrained=True).features[:29]

    def forward(self, x):
        # Store relevant features
        features = []

        # Go through each layer in model, if the layer is in the chosen_features,
        # store it in features. At the end we'll just return all the activations
        # for the specific layers we have in chosen_features
        for layer_num, layer in enumerate(self.model):
            x = layer(x)

            if str(layer_num) in self.chosen_features:
                features.append(x)

        return features

# *Defining Image Upload Function*

In [3]:
import numpy as np
from PIL import Image as PILImage

def load_image(image, device):
    # Gelen resim numpy dizisi şeklinde olabilir
    if isinstance(image, np.ndarray):
        # Numpy dizisini PIL.Image olarak aç
        image = Image.fromarray(image)
    # PIL.Image olarak işlenebilecek bir formatta olmalıdır
    if not isinstance(image, Image.Image):
        raise ValueError("Unsupported image format")

    # PIL.Image'i uygun şekilde işle ve dönüştür
    # Burada uygun işlemleri gerçekleştir (örneğin boyut değişimi, normalizasyon vs.)

    # Dönüştürülen görüntüyü uygun cihaza taşı
    image = loader(image).unsqueeze(0)
    return image.to(device)

# *Transforming to a Tensor and Load Tensor to GPU*

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
imsize = 256

# Here we may want to use the Normalization constants used in the original
# VGG network (to get similar values net was originally trained on), but
# I found it didn't matter too much so I didn't end of using it. If you
# use it make sure to normalize back so the images don't look weird.
loader = transforms.Compose(
    [
        transforms.Resize((imsize, imsize)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# *Image Uploading*

In [ ]:
import os
import gradio as gr
from gradio import components
#from gradio.outputs import Image as OutputImage
from torchvision.transforms import ToPILImage

# NST işlemlerini gerçekleştiren fonksiyon
def neural_style_transfer(original_img, style_img, total_steps, learning_rate, alpha, beta):

    # Yükleme işlemlerini buraya taşıyabilirsiniz
    original_img = load_image(original_img, device)
    style_img = load_image(style_img, device)

    # Yüklenen resimleri uygun cihaza taşı
    #original_img = load_image(original_img, device)
    #style_img = load_image(style_img, device)

    # Generative image'i oluştururken kullanılacak başlangıç resmini orijinal resimden al
    generated = original_img.clone().requires_grad_(True)

    # NST modelini çağır ve değerlendirme modunda kullan
    model = VGG().to(device).eval()

    # Optimizasyon algoritması ve hiperparametreler
    #total_steps = 3000
    #learning_rate = 0.001
    #alpha = 0.01
    #beta = 1
    optimizer = optim.Adam([generated], lr=learning_rate)

    # Kayıp değerlerini saklamak için liste
    total_loss_list = []
    style_loss_list = []
    content_loss_list = []

    # NST algoritmasını uygula
    for step in tqdm(range(total_steps)):
        # Convolution özelliklerini belirli katmanlardan al
        generated_features = model(generated)
        original_img_features = model(original_img)
        style_features = model(style_img)

        # Kayıp başlangıçta 0
        style_loss = original_loss = 0

        # Seçilen katmanlar için tüm özellikleri dolaş
        for gen_feature, orig_feature, style_feature in zip(
            generated_features, original_img_features, style_features
        ):
            # batch_size burada 1 olacak
            batch_size, channel, height, width = gen_feature.shape
            original_loss += torch.mean((gen_feature - orig_feature) ** 2)
            # Oluşturulan için Gram Matrisi hesapla
            G = gen_feature.view(channel, height * width).mm(
                gen_feature.view(channel, height * width).t()
            )
            # Stil için Gram Matrisi hesapla
            A = style_feature.view(channel, height * width).mm(
                style_feature.view(channel, height * width).t()
            )
            style_loss += torch.mean((G - A) ** 2)

        total_loss = alpha * original_loss + beta * style_loss
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
    """
        # Kayıp değerlerini sakla
        total_loss_list.append(total_loss.item())
        style_loss_list.append(style_loss.item())
        content_loss_list.append(original_loss.item())

        fig, axs = plt.subplots(3, figsize=(8, 6))

        # Toplam kayıp değerlerini grafiğe ekleyin
        axs[0].plot(total_loss_list, label='Total Loss')
        axs[0].set_xlabel('Step')
        axs[0].set_ylabel('Total Loss')
        axs[0].set_title('Total Loss Değişimi')
        axs[0].legend()

        # Style loss değerlerini grafiğe ekleyin
        axs[1].plot(style_loss_list, label='Style Loss', color='orange')
        axs[1].set_xlabel('Step')
        axs[1].set_ylabel('Style Loss')
        axs[1].set_title('Style Loss Değişimi')
        axs[1].legend()

        # Content loss değerlerini grafiğe ekleyin
        axs[2].plot(content_loss_list, label='Content Loss', color='green')
        axs[2].set_xlabel('Step')
        axs[2].set_ylabel('Content Loss')
        axs[2].set_title('Content Loss Değişimi')
        axs[2].legend()

        plt.tight_layout()

        plt.savefig("plot.png", format='png')
        # Dosyayı PILImage olarak yükle
        pil_image = Image.open("plot.png")
        # Dosyayı sildikten sonra
        os.remove("plot.png")
        # Resmi döndürün
        return pil_image
    """


    if step % 250 == 0:
        tqdm.write(f"Step {step}, Total Loss: {total_loss.item()}")

    generated_pil_image = ToPILImage()(generated.squeeze().detach())
    generated_pil_image.resize((256,256))
    generated_pil_image.save("generated_image.jpeg", format="jpeg")  # Resmi .jpeg formatında kaydedin

    # Sonuç olarak üretilen resmi döndür
    return generated_pil_image

    # Sonuç olarak üretilen resmi döndür
    #return generated


# Gradio UI'sını oluştur
inputs = [

    components.Image(label="Orijinal Resim", type="pil", sources="upload"),
    components.Image(label="Stil Resmi", type="pil", sources="upload"),
    components.Number(label="Toplam Adım Sayısı", value=3000, minimum=500, maximum=10000, step=10),
    components.Slider(label="Öğrenme Oranı", value=0.001, minimum=0.0001, maximum=0.01, step=0.0001),
    components.Slider(label="Alpha", value=0.01, minimum=0.001, maximum=0.1, step=0.001),
    components.Slider(label="Beta", value=1, minimum=0.1, maximum=10, step=0.1)
]

outputs = [
    components.Image(label="Üretilen Resim", type="pil"),
    #components.Image(label="Grafik", type="pil"),
    ]

# UI ile NST fonksiyonunu bağla ve sunucu modunda başlat
gr.Interface(
    fn=neural_style_transfer,
    inputs=inputs,
    outputs=outputs,
    title="Neural Style Transfer",
    description="Orijinal resmi ve bir stil resmi yükleyin, sonucu görselleştirin.",
).queue().launch(debug = True, share= True)

while True:
    pass

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://eb4272bd0cd4a4161c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# Orijinal resmi yükle
print("Lütfen orijinal resmi yükleyin:")
uploaded_original = files.upload()
original_img_name = list(uploaded_original.keys())[0]

# Stil resmini yükle
print("\nLütfen stil resmini yükleyin:")
uploaded_style = files.upload()
style_img_name = list(uploaded_style.keys())[0]


"""
# Orijinal ve stil resimlerini belirle
original_img = load_image(original_img_name, device)
style_img = load_image(style_img_name, device)

# Generative image'i oluştururken kullanılacak başlangıç resmini orijinal resimden al
generated = original_img.clone().requires_grad_(True)
"""

#generated = torch.randn(original_img.shape, device=device, requires_grad=True)

# *Defining Hyperparameters & Optimizing*

In [ ]:
model = VGG().to(device).eval()

# Hyperparameters
total_steps = 1500
learning_rate = 0.001
alpha = 0.01
beta = 1
optimizer = optim.Adam([generated], lr=learning_rate)

In [ ]:
total_loss_list = []  # Toplam kayıp değerlerini saklamak için liste
style_loss_list = []  # Style loss değerlerini saklamak için liste
content_loss_list = []  # Content loss değerlerini saklamak için liste

for step in tqdm(range(total_steps)):
    # Obtain the convolution features in specifically chosen layers
    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)

    # Loss is 0 initially
    style_loss = original_loss = 0

    # iterate through all the features for the chosen layers
    for gen_feature, orig_feature, style_feature in zip(
        generated_features, original_img_features, style_features
    ):

        # batch_size will just be 1
        batch_size, channel, height, width = gen_feature.shape
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)
        # Compute Gram Matrix of generated
        G = gen_feature.view(channel, height * width).mm(
            gen_feature.view(channel, height * width).t()
        )
        # Compute Gram Matrix of Style
        A = style_feature.view(channel, height * width).mm(
            style_feature.view(channel, height * width).t()
        )
        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    total_loss_list.append(total_loss.item())  # Toplam kayıp değerini listeye ekle
    style_loss_list.append(style_loss.item())  # Style loss değerini listeye ekle
    content_loss_list.append(original_loss.item())  # Content loss değerini listeye ekle

    if step % 250 == 0:
        tqdm.write(f"Step {step}, Total Loss: {total_loss.item()}")
        #save_image(generated, "generated.png")




# Toplam kayıp değerlerinin grafiği
plt.plot(total_loss_list, label='Total Loss')
plt.xlabel('Step')
plt.ylabel('Total Loss')
plt.title('Total Loss Değişimi')
plt.legend()
plt.show()

# Style loss değerlerinin grafiği
plt.plot(style_loss_list, label='Style Loss', color='orange')
plt.xlabel('Step')
plt.ylabel('Style Loss')
plt.title('Style Loss Değişimi')
plt.legend()
plt.show()

# Content loss değerlerinin grafiği
plt.plot(content_loss_list, label='Content Loss', color='green')
plt.xlabel('Step')
plt.ylabel('Content Loss')
plt.title('Content Loss Değişimi')
plt.legend()
plt.show()



save_image(generated, "generated.png")

In [ ]:
from skimage.metrics import structural_similarity as ssim
import numpy as np

# Örnek olarak, orijinal ve üretilen görüntülerin numpy dizileri olarak temsili
original_image_np = original_img.squeeze(0).cpu().detach().numpy().transpose(1, 2, 0)
generated_image_np = generated.squeeze(0).cpu().detach().numpy().transpose(1, 2, 0)

# SSIM hesaplama
ssim_value = ssim(original_image_np, generated_image_np, multichannel=True)

print("SSIM:", ssim_value)
